### [Knock Knock Server](https://docs.oracle.com/javase/tutorial/networking/sockets/clientServer.html)
Knock Knock Server实现了服务端和客户端的一段对话joke. 对话如下
```
Server: "Knock knock!"
Client: "Who's there?"
Server: "Dexter."
Client: "Dexter who?"
Server: "Dexter halls with boughs of holly."
Client: "Groan."
```
#### Server端
1. 绑定端口开启服务  
2. accept()接受连接请求,建立socket连接  
3. 从socket连接中获得输入输出流,并包装成Reader,Writer  
4. 根据输入返回回应

In [ ]:
import java.io.BufferedReader;
import java.io.IOException;
import java.io.InputStreamReader;
import java.io.PrintWriter;
import java.net.ServerSocket;
import java.net.Socket;

public class KnockKnockServer {
    public static void main(String[] args) {
        int portNumber = 4444;
        try {
            ServerSocket ss = new ServerSocket(portNumber);
            Socket clientSocket = ss.accept();
            BufferedReader reader = new BufferedReader(new InputStreamReader(clientSocket.getInputStream()));
            PrintWriter writer = new PrintWriter(clientSocket.getOutputStream(), true);

            String outputLine = null;
            String inputLine = null;

            KnockKnockProtocol kkp = new KnockKnockProtocol();
            outputLine = kkp.processInput(null);
            writer.println(outputLine);
            System.out.println("server response:"+outputLine);

            while((inputLine=reader.readLine())!=null){
                outputLine=kkp.processInput(inputLine);
                System.out.println("Server receive:"+inputLine);
                writer.println(outputLine);
                System.out.println("server response:"+outputLine);
                if(outputLine.equalsIgnoreCase("bye."))
                    break;
            }

        } catch (IOException e) {
            System.out.println("Exception caught when trying to listen on port "
                    + portNumber + " or listening for a connection");
            System.out.println(e.getMessage());
        }

    }
}

#### KnockKnockProtocol
处理针对提问的回答

In [ ]:
public class KnockKnockProtocol {
    private static final int WAITING = 0;
    private static final int SENTKNOCKKNOCK = 1;
    private static final int SENTCLUE = 2;
    private static final int ANOTHER = 3;

    private static final int NUMJOKES = 5;

    private int state = WAITING;
    private int currentJoke = 0;

    private String[] clues = { "Turnip", "Little Old Lady", "Atch", "Who", "Who" };
    private String[] answers = { "Turnip the heat, it's cold in here!",
            "I didn't know you could yodel!",
            "Bless you!",
            "Is there an owl in here?",
            "Is there an echo in here?" };

    public String processInput(String theInput) {
        String theOutput = null;

        if (state == WAITING) {
            theOutput = "Knock! Knock!";
            state = SENTKNOCKKNOCK;
        } else if (state == SENTKNOCKKNOCK) {
            if (theInput.equalsIgnoreCase("Who's there?")) {
                theOutput = clues[currentJoke];
                state = SENTCLUE;
            } else {
                theOutput = "You're supposed to say \"Who's there?\"! " +
                        "Try again. Knock! Knock!";
            }
        } else if (state == SENTCLUE) {
            if (theInput.equalsIgnoreCase(clues[currentJoke] + " who?")) {
                theOutput = answers[currentJoke] + " Want another? (y/n)";
                state = ANOTHER;
            } else {
                theOutput = "You're supposed to say \"" +
                        clues[currentJoke] +
                        " who?\"" +
                        "! Try again. Knock! Knock!";
                state = SENTKNOCKKNOCK;
            }
        } else if (state == ANOTHER) {
            if (theInput.equalsIgnoreCase("y")) {
                theOutput = "Knock! Knock!";
                if (currentJoke == (NUMJOKES - 1))
                    currentJoke = 0;
                else
                    currentJoke++;
                state = SENTKNOCKKNOCK;
            } else {
                theOutput = "Bye.";
                state = WAITING;
            }
        }
        return theOutput;
    }
}


#### Client端
1. 请求服务器建立socket连接
2. 从socket连接中获取输入输出流,并包装成Reader,Writer
3. 获取键盘输入流,并包装成Reader
4. 把键盘输入发送给Server,等待对话回复
```  
Server: Knock! Knock!
键盘输入: Who's there?
Client: Who's there?
Server: Turnip
键盘输入: Turnip who?
Client: Turnip who?
Server: Turnip the heat, it's cold in here! Want another? (y/n)   
```
If you want to hear another joke, type y; if not, type n. If you type y, the server begins again with "Knock! Knock!" If you type n, the server says "Bye." thus causing both the client and the server to exit.

In [ ]:
import java.io.BufferedReader;
import java.io.IOException;
import java.io.InputStreamReader;
import java.io.PrintWriter;
import java.net.Socket;
import java.net.UnknownHostException;

public class KnockKnockClient {
    public static void main(String[] args) {
        int port = 4444;

        try (
            Socket socket = new Socket("localhost", port);
            PrintWriter writer = new PrintWriter(socket.getOutputStream(), true);
            BufferedReader reader = new BufferedReader(new InputStreamReader(socket.getInputStream()));
        ) {
            String fromServer = null;
            String fromUser = null;
            BufferedReader stdIn = new BufferedReader(new InputStreamReader(System.in));
            while ((fromServer=reader.readLine())!=null){
                System.out.println("from server:"+fromServer);
                if(fromServer.equalsIgnoreCase("bye"))
                    break;
                fromUser = stdIn.readLine();
                if(fromUser!=null){
                    System.out.println("Client:"+fromUser);
                    writer.println(fromUser);
                }
            }
        } catch (UnknownHostException e) {
            e.printStackTrace();
        } catch (IOException e) {
            e.printStackTrace();
        }
    }
}
